## Clinical Note Synopsis & Preventive Health Recommendations


After reading a clinic note, we show various methods of extracting the data including:  
- Read Clinic Note
- Identify Components of Note  
    - [Identify any positive ROS findings](#identify-any-positive-ros-findings)
    - [Identify Patient's Medications - and the condition being treated](#identify-patient's-medications---and-the-condition-being-treated)
    - [Highlight MEDICATIONS in the note](#highlight-medications-in-the-note)
- [Identify Medical Considerations based on Clinical Note](#identify-medical-considerations-based-on-clinical-note)
    - [Determine 5-7 Medical Consideration (one or two words in length) ](#determine-5-7-medical-consideration-(one-or-two-words-in-length))
    - [Highlight the sentence that indicates the Medical Consideration](#highlight-the-sentence-that-indicates-the-medical-consideration)
- [Create Assessment and Plan](#create-assessment-and-plan)
    - [Summarize Note - Produce Assessment and Plan](#summarize-note---produce-assessment-and-plan)
   - [Produce Assessment and Plan (SOAP format)](#produce-assessment-and-plan-(soap-format))
   - [Produce Assessment and Plan as a Summary Paragraph](#produce-assessment-and-plan-as-a-summary-paragraph)
- [Preventive Care Recommendations](#preventive-care-recommendations)
   - [Show Specific Reasons for Recommendations](#show-specific-reasons-for-recommendations)
   - [Preventive Care Recommendations - Summary Description](#preventive-care-recommendations---summary-description)
   - [Preventive Care Recommendations - Personalized for the Patient](#preventive-care-recommendations---personalized-for-the-patient)
    - [Website links to Preventive Care Recommendations](#website-links-to-preventive-care-recommendations)
    - [Translate into Spanish and Vietnamese](#translate-into-spanish-and-vietnamese)



In [9]:
import os
import re
import openai
import webbrowser
import datetime 
from datetime import datetime
import json
from json import loads, dumps

import csv
import pandas as pd

import ast 
import subprocess
from CSV_to_JSON_or_DICT import CSV_to_DICT


from chatGPT_utilities import setup_OpenAI_API
from chatGPT_utilities import json_to_dict
from chatGPT_utilities import pp_json
from chatGPT_utilities import get_completion
from chatGPT_utilities import get_completion_from_messages_FULL
from chatGPT_utilities import get_completion_from_messages
from chatGPT_utilities import read_file
from chatGPT_utilities import token_usage
from chatGPT_utilities import print_token_usage
from chatGPT_utilities import openai_object_to_dict
from chatGPT_utilities import CSV_to_DICT


from chatGPT_utilities import replace_with_bold_red
from chatGPT_utilities import write_to_file
from chatGPT_utilities import capitalize_all_letters
from chatGPT_utilities import color_code_clinic_note
from chatGPT_utilities import convert_response_STR_to_DICT
from chatGPT_utilities import highlight_text


openai.api_key = setup_OpenAI_API()
starting_directory = os.getcwd()

starting_directory /Users/michael/Coding
Directory:/Users/michael/Coding


### Function - Get Completion From Messages Full And Costs

In [43]:
number = 10004
print(f'Completion tokens: {number:10,.0f}')
print('\033[91m\033[1m' + '\n\nToken Usage:'+ '\033[0m')



Completion tokens:     10,004


Token Usage:


In [44]:
def print_token_usage(response_details,cost_per_unit = 0.002, number_tokens_per_unit = 1000):
    try:
        # print(response.choices[0].message["content"])
        print('\033[91m\033[1m' + '\n\nToken Usage:'+ '\033[0m')
        print(f'Cost to run query is based on token usage, which is $0.002 / 1K tokens')
        print(f'Completion tokens: {response_details.usage["completion_tokens"]:10,.0f}')
        print(f'Prompt tokens    : {response_details.usage["prompt_tokens"]:10,.0f}')
        print(f'Total tokens     : {response_details.usage["total_tokens"]:10,.0f}')
    except:
        print(f'The response does not contain any token usage data')
        print(f'This is because we used "get_completion_from_messages"')
        print(f'If you want token usage data you must use "get_completion_from_messages_FULL"')
    cost_of_tokens = cost_per_unit * (response_details.usage["total_tokens"]/number_tokens_per_unit)
    cost_of_query = round(cost_of_tokens,3)
    return cost_of_query


In [45]:
def get_completion_from_messages_FULL_and_costs():
    start_time = datetime.now() 

    messages = [{"role": "user", "content": prompt}]
    response_details = get_completion_from_messages_FULL(messages)
    response = response_details.choices[0].message["content"]
    end_time = datetime.now() 
    print(response, '\n'*2)
    print('end_time  :', end_time.strftime("%H:%M:%S"))
    elapsed_time = end_time - start_time
    print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
    cost = print_token_usage(response_details)
    print(f'Cost to run query: ${cost}')
    return response

In [2]:
url_chatGPT_usage = 'https://platform.openai.com/account/usage'
_ = webbrowser.open_new(url_chatGPT_usage)

# Globals

In [11]:
list_note_sections =   ['Patient Information','Name','Age','Ethnicity',
                        'Gender','Reason for Visit',
                        'History of Present Illness',
                        'Medical History','Family History','Social History',
                        'Review of Systems','Physical Examination',
                        'Assessment and Plan']

# Read Clinic Note

In [12]:
dir_patient_notes = 'Sample_Patient_Notes'
file_name = 'Maria_Sanchez.txt'
patient_note = read_file(os.path.join(dir_patient_notes,
                                     file_name))
patient_note_WITHOUT_A_and_P = patient_note.split('Assessment and Plan')[0]
patient_note_for_display = color_code_clinic_note(patient_note_WITHOUT_A_and_P,
                                                  list_note_sections,
                                                 'Red')
print(patient_note_for_display)

Patient Information:
Name: Maria Sanchez
Age: 67
Gender: Female
Ethnicity: Hispanic
Address : Sacramento CA

Reason for Visit: Annual Physical

History of Present Illness:
The patient, a 67-year-old white female, presents today for her annual physical examination. She has not reported any specific complaints or concerns at this time.

Medical History:
The patient has a history of hypertension, which is well-controlled with enalapril and Hydralazine
She also has a history of hyperlipidemia and is on a simvastatin to manage her cholesterol levels.
There is no known history of diabetes or thyroid disorders.
The patient has no significant surgical history.


Family History:
The patient reports a family history of cardiovascular disease, with her father having a heart attack in his late 60s. Her mother is alive and in good health. Mother had colon cancer but is currently in remission. Her sister had ALS and died at age 35.

Social History:
The patient smokes 1/2 ppd. She has a glass of wine


<a id="identify-any-positive-ros-findings"></a>
## Identify any positive ROS findings

In [19]:
prompt = f"""
As a medical chart abstractor, your task is to extract relevant information from \ 
a medical patient's clinic note. 


Identify any POSITIVE findings in the Review of Systems.
Format your response as a numbered list of items, formatted as follows:
ROS Positive findings:
1. 
2. 

N. 

Review: ```{patient_note_WITHOUT_A_and_P}```
"""
response = get_completion(prompt)
print(response)

ROS Positive findings:
1. Occasional urinary frequency
2. Mild rash on forearm



<a id="identify-patient's-medications---and-the-condition-being-treated"></a>
## Identify Patient's Medications - and the condition being treated


In [20]:
prompt = f"""

From the note which is delimited by triple backticks, 
extract the names of the medications that the patient is taking and for what medical conditions they are
taking them

Format your response as follows:

CURRENT MEDICATIONS:
1. medication is being taken for ...
2. medication is being taken for ...

N. medication is being taken for ...

Review: ```{patient_note_WITHOUT_A_and_P}```
"""
response = get_completion(prompt)
print(response)

CURRENT MEDICATIONS:
1. Enalapril - medication is being taken for hypertension
2. Hydralazine - medication is being taken for hypertension
3. Simvastatin - medication is being taken to manage hyperlipidemia




<a id="highlight-medications-in-the-note"></a>
## Highlight MEDICATIONS in the note
- Medications are highlighted in green<span style="color:green"> **CAPITAL** </span> letters.

In [17]:
prompt = f"""

From the note which is delimited by triple backticks, 
extract the names of the medications that the patient is taking. 

Return the names of the medications in a python list as in the following example:
['Tylenol','aspirin']

Review: ```{patient_note_WITHOUT_A_and_P}```
"""
response = get_completion(prompt)
list_pt_meds = ast.literal_eval(response)
patient_note_for_display = color_code_clinic_note(patient_note_WITHOUT_A_and_P,
                                                  list_note_sections,
                                                 'Red')
patient_note_for_display = color_code_clinic_note(patient_note_for_display,
                                                  list_pt_meds,
                                                 'Green')
# print(patient_note_for_display)

# patient_note_for_display = patient_note
# finds meds and ignores case
for ele in list_pt_meds:
    pattern = re.compile(re.escape(ele), re.IGNORECASE)
    match = re.search(pattern, patient_note_for_display)
    
    # print(ele, match, match.span()[0],match.span()[1])
    new_ele = patient_note_for_display[match.span()[0]:match.span()[1]]
    patient_note_for_display = \
    patient_note_for_display.replace(new_ele,'\033[92;1m' + 
                                     new_ele.upper() + '\033[0m')
print(patient_note_for_display)

Patient Information:
Name: Maria Sanchez
Age: 67
Gender: Female
Ethnicity: Hispanic
Address : Sacramento CA

Reason for Visit: Annual Physical

History of Present Illness:
The patient, a 67-year-old white female, presents today for her annual physical examination. She has not reported any specific complaints or concerns at this time.

Medical History:
The patient has a history of hypertension, which is well-controlled with ENALAPRIL and HYDRALAZINE
She also has a history of hyperlipidemia and is on a SIMVASTATIN to manage her cholesterol levels.
There is no known history of diabetes or thyroid disorders.
The patient has no significant surgical history.


Family History:
The patient reports a family history of cardiovascular disease, with her father having a heart attack in his late 60s. Her mother is alive and in good health. Mother had colon cancer but is currently in remission. Her sister had ALS and died at age 35.

Social History:
The patient smokes 1/2 ppd. She has a glass of wine




<a id="identify-medical-considerations-based-on-clinical-note"></a>
# Identify Medical Considerations based on Clinical Note

<a id="determine-5-7-medical-consideration-(one-or-two-words-in-length)"></a>
## Determine 5-7 Medical Consideration (one or two words in length)


In [9]:
prompt = f"""
As a medical chart abstractor, your task is to extract relevant information from \ 
a medical patient's clinic note. 


Determine at least 7 medical considerations that are being discussed in the \
clinical note below, which is delimited by triple backticks.

Make each item one or two words long. 
Be sure to look specifically for any concerns about cancer

Format your response as a numbered list of items, formatted as follows:
MEDICAL CONSIDERATIONS:
1. 
2. 

N. 

Review: ```{patient_note_WITHOUT_A_and_P}```
"""
response = get_completion(prompt)
print(response)

MEDICAL CONSIDERATIONS:
1. Hypertension
2. Hyperlipidemia
3. Cardiovascular disease
4. Colon cancer
5. ALS
6. Smoking
7. Mild rash


<a id="highlight-the-sentence-that-indicates-the-medical-consideration"></a>
## Highlight the sentence that indicates the Medical Consideration

In [10]:
prompt = f"""
As a medical chart abstractor, your task is to extract relevant information from \ 
a medical patient's clinic note. 


Determine at least 7 medical considerations that are being discussed in the \
clinical note below, which is delimited by triple backticks.

Make each item one or two words long.  
For each item, list one sentence in the clinic note that indicates
that the item is relevant

Return a dictionary where the item is the key and the value is the exact sentence that refers to the key.

Be sure to look specifically for any concerns about cancer

Review: ```{patient_note_WITHOUT_A_and_P}```
"""
response = get_completion(prompt)

keyword_sentence_dict = convert_response_STR_to_DICT(response) # convert response string to a dict
patient_note_for_display = patient_note_WITHOUT_A_and_P

for key, value in keyword_sentence_dict.items():
    sentence = value
    pertinent_sentence = sentence[0:len(sentence)]
    highlighted_line = highlight_text(pertinent_sentence)
    assert pertinent_sentence in patient_note_for_display, '"' + sentence  + '" is NOT found in NOTE'
    patient_note_for_display = patient_note_for_display.split(pertinent_sentence)[0] + \
                                highlighted_line + \
                                ' [' + key.upper() + ']  ' +  \
                                patient_note_for_display.split(pertinent_sentence)[1]
patient_note_for_display = color_code_clinic_note(patient_note_for_display,
                           list_note_sections,
                          'Red')
print('Medical Considerations:')
list_of_keys = list(keyword_sentence_dict.keys())
for num, name in enumerate(list_of_keys, start=1): 
    print("{0:3}: {1}".format(num, name))

print( '\n'*2)  
explanation = """
For each "Medical Consideration",
we highlight the sentence in yellow - and show the keyword
"""

highlighted_line = highlight_text(explanation)
print(highlighted_line, '\n'*2)  

print(patient_note_for_display)

Medical Considerations:
  1: Hypertension
  2: Hyperlipidemia
  3: Cardiovascular disease
  4: Colon cancer
  5: ALS
  6: Smoking
  7: Alcohol consumption




For each "Medical Consideration",
we highlight the sentence in yellow - and show the keyword
 


Patient Information:
Name: Maria Sanchez
Age: 67
Gender: Female
Ethnicity: Hispanic
Address : Sacramento CA

Reason for Visit: Annual Physical

History of Present Illness:
The patient, a 67-year-old white female, presents today for her annual physical examination. She has not reported any specific complaints or concerns at this time.

Medical History:
The patient has a history of hypertension, which is well-controlled with enalapril and Hydralazine [HYPERTENSION]  
She also has a history of hyperlipidemia and is on a simvastatin to manage her cholesterol levels [HYPERLIPIDEMIA]  .
There is no known history of diabetes or thyroid disorders.
The patient has no significant surgical history.


Family History:
The patient reports a family 

<a id="create-assessment-and-plan"></a>
# Create Assessment and Plan

- Acting as a PCP, read note, extract information
- Produce an assessent and plan
- Include relevant preventive health recommendations

In [46]:
medical_specialist = 'primary care physician'
prompt = f"""
Acting as a {medical_specialist}, your task is to extract relevant information from \ 
a clinical note to produce an assessent and plan for the patient.


Format your response as follows:
Patient Age:
Patient Gender:
CURRENT MEDICATIONS:
1.
2.

N.

MEDICAL CONSIDERATIONS:
1. Medical Concern
    Assessment : 
    Plan       :

2. Medical Concern
    Assessment : 
    Plan       :
    
N. Medical Concern
    Assessment : 
    Plan       :
    
    
Preventive Care Recommendations:
    a.
    b.

Preventive Care Recommendations should include any recommendations about 
screening for cancer if there has been a family history of cancer . 
List any screening recommendations under 'Preventive Care Recommendations' 
and do not include them as a Medical Consideration.
Be sure to mention Education regarding alcohol and tobacco in the preventive care recommendations, if applicable


Review: ```{patient_note}```
"""

# assessment = get_completion(prompt)
assessment = get_completion_from_messages_FULL_and_costs() # prompt is global

# print(assessment)

Patient Age: 67
Patient Gender: Female

CURRENT MEDICATIONS:
1. Enalapril (for hypertension)
2. Hydralazine (for hypertension)
3. Simvastatin (for hyperlipidemia)

MEDICAL CONSIDERATIONS:
1. Hypertension
    Assessment: Hypertension is well-controlled with current medications.
    Plan: Continue current medications and monitor blood pressure regularly.

2. Hyperlipidemia
    Assessment: Hyperlipidemia is managed with simvastatin.
    Plan: Continue simvastatin and monitor lipid levels regularly.

Preventive Care Recommendations:
    a. Schedule routine laboratory investigations, including complete blood count, lipid panel, and comprehensive metabolic panel, to assess overall health status.
    b. Advise the patient to continue current medications and lifestyle modifications to maintain optimal health.
    c. Schedule a follow-up visit to review the results and address any concerns that may arise.

Note: No specific screening recommendations for cancer are mentioned in the clinical note

In [12]:
file_name_generated_assessment = file_name + '_GENERATED_ASSESSMENT'
assessment = 'GENERATED ASSESSMENT and PLAN\n\n\nCURRENT MEDICATIONS' + assessment.split('CURRENT MEDICATIONS')[1]
write_to_file(file_name_generated_assessment,patient_note,assessment)
_ = subprocess.call(['open',   file_name_generated_assessment + '.txt'])



<a id="summarize-note---produce-assessment-and-plan"></a>
## Summarize Note - Produce Assessment and Plan


In [19]:
prompt = f"""
As a medical chart abstractor, your task is to summarize the Assessment and Plan from 
a medical patient's clinic note. 

Review: ```{patient_note_WITHOUT_A_and_P}```
"""
response = get_completion(prompt)
print(response)

Assessment:
- 67-year-old female patient presenting for annual physical examination
- History of hypertension, well-controlled with enalapril and Hydralazine
- History of hyperlipidemia, managed with simvastatin
- No known history of diabetes or thyroid disorders
- No significant surgical history
- Family history of cardiovascular disease, father had a heart attack in his late 60s
- Mother had colon cancer but is currently in remission
- Sister had ALS and died at age 35
- Patient smokes 1/2 ppd and has a glass of wine most evenings
- Retired and leads a moderately active lifestyle
- Balanced diet and no known food allergies
- Mild rash on forearm, no significant lesions or skin changes
- No other significant findings on review of systems or physical examination

Plan:
- Continue current medications for hypertension and hyperlipidemia
- Advise patient to quit smoking and reduce alcohol consumption
- Encourage regular exercise and healthy lifestyle habits
- Monitor rash on forearm and p


<a id="produce-assessment-and-plan-(soap-format)"></a>
## Produce Assessment and Plan (SOAP format)


In [21]:
prompt = f"""
As a medical chart abstractor, your task is to summarize the Assessment and Plan from 
a medical patient's clinic note. 

Construct an assessment and plan in the following format:
Medical Problem # 1
Objective: ...
Assessment : ...
Plan : ...

Medical Problem # 2
Objective: ...
Assessment : ...
Plan : ...

Review: ```{patient_note_WITHOUT_A_and_P}```
"""
response = get_completion(prompt)
color_coded_note = color_code_clinic_note(response,['Medical Problem'],'Red')
color_coded_note = color_code_clinic_note(color_coded_note,['Objective:','Assessment:','Plan:'],'Black')
print(color_coded_note)

Medical Problem #1
Objective: Annual physical examination
Assessment: The patient, a 67-year-old white female, presents for her annual physical examination. She has no specific complaints or concerns at this time.
Plan: Perform a comprehensive physical examination, review medical history, assess vital signs, and order any necessary laboratory tests.

Medical Problem #2
Objective: Hypertension management
Assessment: The patient has a history of hypertension, which is well-controlled with enalapril and Hydralazine.
Plan: Continue current medication regimen, monitor blood pressure regularly, and schedule follow-up appointments to assess the effectiveness of the treatment.

Medical Problem #3
Objective: Hyperlipidemia management
Assessment: The patient has a history of hyperlipidemia and is on simvastatin to manage her cholesterol levels.
Plan: Continue current medication regimen, monitor lipid levels regularly, and provide lifestyle counseling on diet and exercise to further manage hyperl

<a id="produce-assessment-and-plan-as-a-summary-paragraph"></a>
## Produce Assessment and Plan as a Summary Paragraph

In [8]:
prompt = f"""
As a medical chart abstractor, your task is to summarize the Assessment and Plan from 
a medical patient's clinic note. 

Construct a summary, in paragraph form that should be approximately 5 sentences long
Review: ```{patient_note_WITHOUT_A_and_P}```
"""
response = get_completion(prompt)
print(response)

Maria Sanchez, a 67-year-old Hispanic female, presented for her annual physical examination with no specific complaints or concerns. Her medical history includes well-controlled hypertension with enalapril and Hydralazine, as well as hyperlipidemia managed with simvastatin. There is no history of diabetes or thyroid disorders, and she has no significant surgical history. Her family history includes cardiovascular disease in her father, colon cancer in her mother (currently in remission), and ALS in her sister. Maria smokes half a pack of cigarettes per day and has a glass of wine most evenings. She leads a moderately active lifestyle, follows a balanced diet, and has no known food allergies. The review of systems reveals no significant abnormalities, except for a mild rash on her forearm. The physical examination shows normal vital signs, a well-nourished appearance, and no acute distress. Overall, Maria appears to be in good health with no significant findings.


In [79]:
_ = subprocess.call(['open',   os.path.join('Sample_Patient_Notes',file_name) ])



<a id="preventive-care-recommendations"></a>
# Preventive Care Recommendations





<a id="show-specific-reasons-for-recommendations"></a>
## Show Specific Reasons for Recommendations
- Medical Consideration
    - Recommendation
    - Reason for Recommendation

In [53]:
prompt = f"""
Your task is to extract relevant information from \ 
a clinical note to form recommendations regarding preventive healthcare.


Determine 3-5 areas that would be appropriate for the patient described in the clinical note. 
For each recommendation, Show the parts of the 
clinical note that prompt these recommendations

Format your response as follows, with each recommendation separated by a blank line
Preventive Health Recommendations
1. Topic 
    a. brief description on the next line
    b. Section of note that pertains to recommendation 

2. Topic 
    a. brief description on the next line
    b. Section of note that pertains to recommendation 

etc


Review: ```{patient_note}```
"""

recommendations = get_completion(prompt)
print(recommendations)

Preventive Health Recommendations

1. Cardiovascular Health
    a. The patient has a family history of cardiovascular disease, with her father having a heart attack in his late 60s.
    b. Section of note: Family History

2. Cancer Screening
    a. The patient's mother had colon cancer but is currently in remission.
    b. Section of note: Family History

3. Smoking Cessation
    a. The patient smokes 1/2 ppd.
    b. Section of note: Social History

4. Alcohol Consumption
    a. The patient has a glass of wine most evenings.
    b. Section of note: Social History

5. Bone Health
    a. No mention of osteoporosis risk factors, but it is recommended to assess bone health in older adults.
    b. Section of note: Not explicitly mentioned, but relevant for older adults.


<a id="preventive-care-recommendations---summary-description"></a>
## Preventive Care Recommendations - Summary Description
- Extract relevant information to form recommendations regarding preventive healthcare.
- 3-5 areas appropriate for the patient described in the clinical note.



In [52]:
prompt = f"""
Your task is to extract relevant information from \ 
a clinical note to form recommendations regarding preventive healthcare.


Determine 3-5 areas that would be appropriate for the patient described in the clinical note.

Format your response as follows, with each recommendation separated by a blank line
Preventive Health Recommendations
1. Topic:
    Paragraph of 3-5 sentences

2. Topic:
    Paragraph of 3-5 sentences

etc


Review: ```{patient_note}```
"""

recommendations = get_completion(prompt)
print(recommendations)

Preventive Health Recommendations

1. Cardiovascular Health:
    Maria Sanchez has a family history of cardiovascular disease, with her father having a heart attack in his late 60s. Given her age and family history, it is important for Maria to prioritize her cardiovascular health. She should continue to manage her hypertension and hyperlipidemia with medications as prescribed. Additionally, she should consider incorporating regular cardiovascular exercise, such as brisk walking or swimming, into her routine. It is also recommended that Maria undergo regular cardiovascular screenings, including blood pressure checks, lipid panels, and possibly an electrocardiogram (ECG), to monitor her heart health.

2. Cancer Screening:
    Although Maria's mother had colon cancer, she is currently in remission. However, it is still important for Maria to be proactive about her own cancer screenings. As a woman over the age of 50, she should undergo regular screenings for breast cancer and cervical ca

In [16]:
file_name = 'Maria Sanchez_Friday'
write_to_file(file_name,patient_note_cap_version,assessment + '\n\n' + recommendations)
_ = subprocess.call(['open',   file_name + '.txt'])

Mon Jun 26 at 09:48
Maria Sanchez_Friday was created in /Users/michael/Coding


In [21]:
# print(patient_note)

In [22]:
prompt = f"""
Acting as a medical coding expert, extract the patient's name, gender and ethnicity 
and output a dictionary with keys of "name","age", "gender",and "ethnicity"

Review: ```{patient_note}```
"""
patient_details= get_completion(prompt)
print(patient_details)

Output:
{
"name": "Maria Sanchez",
"age": 67,
"gender": "Female",
"ethnicity": "Hispanic"
}




<a id="preventive-care-recommendations---personalized-for-the-patient"></a>
## Preventive Care Recommendations - Personalized for the Patient


In [61]:
prompt = f"""
Your task is to extract relevant information from \ 
a clinical note to form recommendations regarding preventive healthcare.

Determine 3-5 areas that would be appropriate for the patient described in the clinical note.

Format your response as follows, with each recommendation separated by a blank line
Preventive Health Recommendations
1. Topic followed by brief description on the next line

2. Topic followed by brief description on the next line
etc

If a recommendation references smoking cessation,
include the English phone information near where the patient lives which could include any of the following:
Kick it California
•	English:  1-800-300-8086
•	Hispanic:  1-800-600-8191
•	Chinese:  1-800-838-8917
•	Korean:  1-800-556-5564
•	Vietnamese:  1-800-778-8440

Always include the English number but if the patient's name or ethnicity suggests it, please add additional
numbers as appropriate

Use a friendly tone and address the recommendations to the patient, 
using their first name in each recommendation so that 
a personalized message is created

Review: ```{patient_note}```
"""

personalized_recommendations = get_completion(prompt)

patient_name = file_name.split(' ')[0]
personalized_recommendations_green = \
    personalized_recommendations.replace(patient_name,'\033[92;1m' + patient_name.upper() + '\033[0m')
print(personalized_recommendations_green)

Preventive Health Recommendations

1. Cardiovascular Health:
Maria, since you have a family history of cardiovascular disease and your father had a heart attack in his late 60s, it's important to prioritize your cardiovascular health. Along with taking your medications for hypertension and hyperlipidemia, I recommend incorporating regular aerobic exercise into your routine, such as brisk walking or swimming. This will help strengthen your heart and improve your overall cardiovascular health.

2. Smoking Cessation:
Maria, as you mentioned that you smoke half a pack per day, I strongly encourage you to consider quitting smoking. Smoking is a major risk factor for heart disease, lung cancer, and many other health problems. It can be challenging to quit, but there are resources available to support you. I recommend reaching out to Kick it California for assistance. You can call their English helpline at 1-800-300-8086. Quitting smoking will greatly improve your health and reduce your risk 



<a id="translate-into-spanish-and-vietnamese"></a>
## Translate into Spanish and Vietnamese


In [63]:
prompt = f"""
Acting as an interpreter, translate the {recommendations} into Spanish and Vietnamese
is to extract 

Review: ```{personalized_recommendations}```
"""
translation = get_completion(prompt)
print(translation)


# file_name = 'Maria Sanchez_Friday'
# write_to_file(file_name,patient_note_cap_version,assessment + '\n\n' + personalized_recommendations)
# # write_to_file(file_name,patient_note_original,response)
# _ = subprocess.call(['open',   file_name + '.txt'])

Spanish Translation:

Recomendaciones de Salud Preventiva

1. Salud Cardiovascular:
María, dado que tienes antecedentes familiares de enfermedad cardiovascular y tu padre sufrió un ataque al corazón en sus últimos 60 años, es importante que priorices tu salud cardiovascular. Además de tomar tus medicamentos para la hipertensión y la hiperlipidemia, te recomiendo incorporar ejercicio aeróbico regular en tu rutina, como caminar rápido o nadar. Esto ayudará a fortalecer tu corazón y mejorar tu salud cardiovascular en general.

2. Cesación del Tabaco:
María, dado que mencionaste que fumas medio paquete al día, te animo encarecidamente a que consideres dejar de fumar. Fumar es un factor de riesgo importante para enfermedades cardíacas, cáncer de pulmón y muchos otros problemas de salud. Puede ser difícil dejar de fumar, pero hay recursos disponibles para apoyarte. Te recomiendo que te pongas en contacto con Kick it California para obtener ayuda. Puedes llamar a su línea de ayuda en español 



<a id="website-links-to-preventive-care-recommendations"></a>
## Website links to Preventive Care Recommendations


In [66]:

poor_habits_dict = {
    "heart-healthy" : "https://health.ucdavis.edu/blog/good-food/5-heart-healthy-steps-to-work-into-your-diet/2020/02" ,
    "colonoscopy": "https://health.ucdavis.edu/radiology/specialties/colon.html",
    "osteoporosis" : "https://health.ucdavis.edu/news/headlines/rheumatologist-nancy-lane-discusses-latest-research-on-osteoarthritis-and-osteoporosis/2023/02",
    "smoking": "https://health.ucdavis.edu/cancer/support/support-services/tobacco-cessation.html#stoptobaccoprogram",
    "alcohol": "https://health.ucdavis.edu/medicalcenter/cliniclocations/specialtycare/substance-use-addiction-Sacramento-Midtown.html",
    "depression": "https://health.ucdavis.edu/psychiatry/index.html",
    "eating disorder" : "https://health.ucdavis.edu/blog/cultivating-health/eating-disorders-symptoms-types-risks-and-ways-to-seek-help/2023/02"
}
prompt = f"""
Your task is to insert relevant URL web links into  
the recommendations regarding preventive healthcare.

If a recommendation pertains to one of the keys of {poor_habits_dict},
then at the end of the recommendation, 
insert the URL found in the value of the dictionary in the following format:
You can find more information at <URL> at our UC Davis Health System website.

Be sure that key in the dictionary is found in the recommendation before inserting the weblink


Review: ```{personalized_recommendations}```
"""

URLs_inserted_into_personalized_recommendations = get_completion(prompt)
print(URLs_inserted_into_personalized_recommendations)

Preventive Health Recommendations

1. Cardiovascular Health:
Maria, since you have a family history of cardiovascular disease and your father had a heart attack in his late 60s, it's important to prioritize your cardiovascular health. Along with taking your medications for hypertension and hyperlipidemia, I recommend incorporating regular aerobic exercise into your routine, such as brisk walking or swimming. This will help strengthen your heart and improve your overall cardiovascular health. You can find more information at https://health.ucdavis.edu/blog/good-food/5-heart-healthy-steps-to-work-into-your-diet/2020/02 at our UC Davis Health System website.

2. Smoking Cessation:
Maria, as you mentioned that you smoke half a pack per day, I strongly encourage you to consider quitting smoking. Smoking is a major risk factor for heart disease, lung cancer, and many other health problems. It can be challenging to quit, but there are resources available to support you. I recommend reaching o

# Extra - Nothing to Read below this 

Some Markdown text with <span style="color:red">some *red* text</span>.

<div class="alert alert-block alert-danger">
There is an issue if the sentence is a compound thought - smokes 1/2 ppd and has a glass of wine. as opposed to 2 separate sentences
</div>

 

In [69]:


# start_time = datetime.now() 

# messages = [{"role": "user", "content": prompt}]
# response_details = get_completion_from_messages_FULL(messages)
# b = response_details.choices[0].message["content"]
#     # print(response)

# # response = get_completion(prompt)
# end_time = datetime.now() 
# print(response, '\n'*2)
# print('end_time  :', end_time.strftime("%H:%M:%S"))
# elapsed_time = end_time - start_time
# print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
# cost = print_token_usage(response_details)
# print(f'Cost to run query: ${cost}')

In [ ]:
clinical_note ="""Medical History:

Name: Maria Sanchez
Age: 67
Gender: Female
Ethnicity: Hispanic


The patient has a history of hypertension, which is well-controlled with medication (name of medication).
She also has a history of hyperlipidemia and is on a statin medication (name of medication) to manage her cholesterol levels.
There is no known history of diabetes or thyroid disorders.
The patient has no significant surgical history.
Family History:
The patient reports a family history of cardiovascular disease, with her father having a heart attack in his late 60s. Her mother is alive and in good health. There is no family history of cancer or other major illnesses.

Social History:
The patient is a non-smoker and does not consume alcohol excessively. She is retired and leads a moderately active lifestyle, engaging in regular walking and light exercises. She follows a balanced diet and has no known food allergies.

"""

## otolaryngologist

In [ ]:
    
# def extract_info_as_a(physician):
#     prompt = f"""
#     Acting as a {medical_specialist}, your task is to extract relevant information from \ 
#     a clinical note to produce an assessent and plan for the patient.


#     Format your response as follows:
#     Patient Age:
#     Patient Gender:
#     CURRENT MEDICATIONS:
#     1.
#     2.

#     N.

#     MEDICAL CONSIDERATIONS:
#     1. Medical Concern
#         Assessment : 
#         Plan       :

#     2. Medical Concern
#         Assessment : 
#         Plan       :

#     N. Medical Concern
#         Assessment : 
#         Plan       :


#     Preventive Care Recommendations:
#         a.
#         b.

#     Preventive Care Recommendations should include any recommendations about 
#     screening for cancer  . 
#     List any screening recommendations under 'Preventive Care Recommendations' 
#     and do not include them as a Medical Consideration.
#     Be sure to mention Education regarding alcohol and tobacco in the preventive care recommendations, if applicable
    
#     Do not refer patient to a doctor or specialist that is in the same specialty as a {medical_specialist}.

#     Review: ```{patient_note}```
#     """
#     return prompt

In [58]:
medical_specialist = 'otolaryngologist'
the_prompt = extract_info_as_a(medical_specialist)
print(the_prompt.split('Review: ')[0])
assessment = get_completion(the_prompt)
print(assessment)


    Acting as a otolaryngologist, your task is to extract relevant information from \ 
    a clinical note to produce an assessent and plan for the patient.


    Format your response as follows:
    Patient Age:
    Patient Gender:
    CURRENT MEDICATIONS:
    1.
    2.

    N.

    MEDICAL CONSIDERATIONS:
    1. Medical Concern
        Assessment : 
        Plan       :

    2. Medical Concern
        Assessment : 
        Plan       :

    N. Medical Concern
        Assessment : 
        Plan       :


    Preventive Care Recommendations:
        a.
        b.

    Preventive Care Recommendations should include any recommendations about 
    screening for cancer  . 
    List any screening recommendations under 'Preventive Care Recommendations' 
    and do not include them as a Medical Consideration.
    Be sure to mention Education regarding alcohol and tobacco in the preventive care recommendations, if applicable
    
    Do not refer patient to a doctor or specialist that is in t

In [ ]:
# dir_patient_notes = 'Sample_Patient_Notes'
# file_name = 'Maria_Sanchez.txt'
# file_name = 'ENT_Patient_no_assesment.txt'


# patient_note = read_file(os.path.join(dir_patient_notes,
#                                      file_name))


# patient_note_original = patient_note

# patient_note_cap_version = patient_note_original
# list_substrings = ['Patient Information','Reason for Visit','Medical History',
#                   'Family History', 'Social History','Review of Systems',
#                   'Physical Examination','Assessment and Plan:']
# for substring in list_substrings:
#     patient_note_cap_version  
#     capitalized_string = capitalize_all_letters(substring)
#     patient_note_cap_version = patient_note_cap_version.replace(substring,capitalized_string)
# # print('\n'*5 ) 
# # print(patient_note_cap_version)


# list_substrings = ['Patient Information','Reason for Visit','Medical History','History of Present Illness',
#                   'Family History', 'Social History','Review of Systems',
#                   'Physical Examination','Assessment and Plan:']
# for substring in list_substrings:
#     index = patient_note.find(substring)
#     patient_note  = replace_with_bold_red(patient_note,  substring)
# print(patient_note)

In [3]:
# def replace_with_bold_red(string, substring):
#     """
#     : Replace substring with BOLD RED in the string
#     """
#     string = string.replace(substring,'\033[91m'+\
#                                      '\033[1m' + '\n\n' + substring + '\033[0m') 
#     return string

# def write_to_file(file_name,patient_note,response):
#     file = open(file_name + '.txt' ,'w') 
#     now = datetime.now()
#     formatted_date = now.strftime("%a %b %d at %H:%M") #print(f'{formatted_date}')
#     file.write('Created on: ' + formatted_date + '\n'*2 )
#     file.write(patient_note + '\n'*2 )
#     file.write(response + '\n'*2 ) #print(f'{file_name} was created in {os.getcwd()}')
#     file.close() 
#     return


# def capitalize_all_letters(string):
#     capitalized_string = ""
#     for char in string:
#         if char.isalpha():
#             capitalized_string += char.upper()
#         else:
#             capitalized_string += char
#     return capitalized_string


# def color_code_clinic_note(string,
#                            list_substrings,
#                           color):
#     """
#     : Input   : pt_note, which is a text string 
#                 list_substrings, which are substrings
#     : Output  :
#     """
#     printing_colors = { 'Red' : '\033[91;1m',
#                    'Green' : '\033[92;1m',
#                     'Brown' : '\033[38;5;94;1m',
#                     'Blue' :  '\033[38;5;18;1m'  ,
#                     'Orange' : '\033[38;5;208;1m',
#                     'Black': '\033[255;0;0;1m',
#                     'Magenta' : '\033[38;5;90m',
#                     'Purple' : '\033[38;5;54m'}


    
   
#     assert color in printing_colors.keys(), \
#     color +  ' is NOT a recognized color for "color_code_clinic_note"'

#     string_for_display = string
#     for substr in list_substrings:
#         # patient_note_for_display = patient_note_for_display.replace(substr,
#         #                                                             '\033[91m\033[1m' + substr + '\033[0m')
#         string_for_display = string_for_display.replace(substr,
#                                   printing_colors[color] + substr +  '\033[0m' )
#     return string_for_display

# def highlight_text(text):
#     YELLOW_BACKGROUND = '\033[43m'
#     RESET_FORMATTING = '\033[0m'
#     return YELLOW_BACKGROUND + text + RESET_FORMATTING

# def convert_response_STR_to_DICT(response):
#     clean_string = response.replace('\n', '') # Remove unnecessary characters ('\n') from the string
#     data_dict = json.loads(clean_string)  # Convert the cleaned string to a dictionary
#     return data_dict

